In [11]:
import pandas as pd
import seaborn as sns
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
df = pd.read_excel("Case_Study_Dataset.xlsx", header=1)
df.head(5)

,hubspot_deal,deal_value,brand_origin,brand_tier,brand_type,products_category,production_origin,sales_channels,instagram_followers,main_markets,ecommerce_platform,items_per_season,creation_date,estimated_close_date,close_date,pipeline,hubspot_stage
0,46,1920.0,Portugal,S,Other,Accessories,Portugal,Ecommerce,10455,Europe,Other,3200,20180110.0,2017-12-15 00:00:00.000000,NaN,Conversion Pipeline,8
1,5714,NaN,Sweden,NaN,Adult,Apparel,"Portugal, Turkey",Ecommerce;Wholesale;Marketplace,NaN,Germany;Europe,Shopify,NaN,20191115.0,2019-12-09 00:00:00.000000,2019-12-09 00:00:00.000000,Onboarding Pipeline,31
2,54,NaN,Portugal,NaN,Other,Accessories,portugal,Ecommerce;Wholesale,5k - 20k,NaN,Shopify,NaN,20180116.0,2019-01-31 15:37:10.000000,NaN,Inside Sales Pipeline,26
3,356,2400.0,Singapore,S,Other,Accessories,Portugal,Wholesale;Ecommerce,N.D.,Middle East,Shopify,400,20180730.0,2018-08-29 17:40:15.000000,NaN,Conversion Pipeline,23
4,58,NaN,Spain,NaN,Men;Women,Footwear,spain,Ecommerce;Own Stores,> 100k,Europe,Shopify,N.D,20180122.0,NaN,NaN,Fairs Pipeline,27


In [3]:
df.shape

(6164, 17)

# Data Cleaning of the Brand Origin

In [4]:
df.brand_origin = df.brand_origin.apply(lambda x: x.lower() if type(x)==str else x) # Capitalize all entries
sorted([x for x in df.brand_origin.unique() if type(x)==str])

['amsterdam',
 'andorra',
 'argentina',
 'australia',
 'australia, new zealand',
 'austria',
 'bahrain',
 'bangladesh',
 'belarus',
 'belgium',
 'belgium-ukraine',
 'berlin',
 'brasil',
 'brazil',
 'bulgaria',
 'california',
 'canada',
 'china',
 'colombia',
 'croatia',
 'cyprus',
 'czech republic',
 'danimarca',
 'denmark',
 'deutschland',
 'dubai',
 'egypt',
 'espanha',
 'estonia',
 'faroe islands',
 'finland',
 'france',
 'france / morocco',
 'france, australia',
 'georgia',
 'germany',
 'germany, netherlands',
 'ghana',
 'grecia',
 'greece',
 'helsinki',
 'holand',
 'hong kong',
 'hungary',
 'iceland',
 'india',
 'indonesia',
 'ireland',
 'island',
 'israel',
 'italy',
 'itália',
 'japan',
 'korea',
 'latvia',
 'letonia',
 'lithuania',
 'luxembourg',
 'malaysia',
 'mexico',
 'moldova',
 'monaco',
 'nepal',
 'netherlands',
 'netherlands (the)',
 'netherlands, turkey',
 'new york',
 'new zealand',
 'norway',
 'partner  referral',
 'peru',
 'poland',
 'poland ',
 'polland',
 'portugal

# **First design question:** What do we want to do with assets with multiple countries? We could duplicate the lines so that it takes that into account, use a single (more important) country or consider it as a separate entity. For now I will evolve with the latter.

In [5]:
# Correcting misspelings or other issues
# One of the entries is island. Could they mean Iceland?
country_manual_corrections = {
 'brasil':'brazil',
 'danimarca': 'denmark',
 'deutschland': 'germany',
 'dubai': 'united arab emirates', # I assume we don't want to separate dubai from other places
 'espanha': 'spain',
 'grecia': 'greece',
 'helsinki': 'finland',
 'holand': 'netherlands',
 'itália':'italy',
 'letonia': 'lithuania',
 'the netherlands': 'netherlands',
 'netherlands (the)': 'netherlands',
 'new york': "usa",
 'poland ':'poland',
 'polland':'poland',
 'partner  referral': 'referral',
 'singapure':'singapore',
 'korea': 'south korea', # I am assuming korea means south korea and never north korea, due to geopolitics and all
 'spain / gran canaria': 'spain',
 'stockholm': 'sweden',
 'suiça': 'switzerland',
 'sweden ': 'sweden',
 'swenden': 'sweden',
 'the netherlands': "netherlands",
 'the netherlands ': "netherlands",
 'turquey': "turkey",
 'uk/ portugal': 'uk portugal',
 'united kingdown':'united kingdom',
 'united kingom': 'united kingdom',
 'united states': 'usa',
 'united states of america': 'usa',
}

In [6]:
df.brand_origin = df.brand_origin.replace(country_manual_corrections)
sorted([x for x in df.brand_origin.unique() if type(x)==str])

['amsterdam',
 'andorra',
 'argentina',
 'australia',
 'australia, new zealand',
 'austria',
 'bahrain',
 'bangladesh',
 'belarus',
 'belgium',
 'belgium-ukraine',
 'berlin',
 'brazil',
 'bulgaria',
 'california',
 'canada',
 'china',
 'colombia',
 'croatia',
 'cyprus',
 'czech republic',
 'denmark',
 'egypt',
 'estonia',
 'faroe islands',
 'finland',
 'france',
 'france / morocco',
 'france, australia',
 'georgia',
 'germany',
 'germany, netherlands',
 'ghana',
 'greece',
 'hong kong',
 'hungary',
 'iceland',
 'india',
 'indonesia',
 'ireland',
 'island',
 'israel',
 'italy',
 'japan',
 'latvia',
 'lithuania',
 'luxembourg',
 'malaysia',
 'mexico',
 'moldova',
 'monaco',
 'nepal',
 'netherlands',
 'netherlands, turkey',
 'new zealand',
 'norway',
 'peru',
 'poland',
 'portugal',
 'portugal/ united kingdom',
 'referral',
 'romania',
 'russia',
 'saudi arabia',
 'scotland',
 'serbia',
 'singapore',
 'slovakia',
 'slovenia',
 'south africa',
 'south korea',
 'spain',
 'sri lanka',
 'sviz

In [7]:
country_order_counts = pd.DataFrame(data=df.brand_origin.value_counts(normalize=True))
country_order_counts.columns = ["frequency"]
country_order_counts["cumulative frequency"] = country_order_counts.cumsum()
country_order_counts.T

,spain,united kingdom,france,italy,germany,portugal,denmark,uk,netherlands,usa,sweden,belgium,poland,greece,finland,norway,turkey,switzerland,japan,austria,ukraine,australia,canada,lithuania,south korea,latvia,ireland,russia,romania,new zealand,hungary,brazil,iceland,estonia,bulgaria,czech republic,scotland,israel,moldova,serbia,peru,slovakia,china,india,singapore,hong kong,argentina,croatia,taiwan,slovenia,united arab emirates,uk portugal,luxembourg,cyprus,mexico,referral,island,malaysia,georgia,belarus,indonesia,us - eu branch,amsterdam,bahrain,"usa, uk",usa georgia,monaco,"usa, spain",california,egypt,"netherlands, turkey",berlin,united kingdom/usa,colombia,"germany, netherlands",andorra,usa / pt,faroe islands,portugal/ united kingdom,us,"usa, portugal",nepal,"united kingdom, greece",bangladesh,"australia, new zealand",belgium-ukraine,venezuela,south africa,svizzera,france / morocco,sri lanka,thailand,"usa, brasil",ghana,saudi arabia,"france, australia"
frequency,0.106114,0.104261,0.101735,0.094829,0.092976,0.081691,0.056594,0.046488,0.045814,0.035371,0.033182,0.025602,0.020044,0.017854,0.014317,0.010106,0.009938,0.009432,0.008927,0.007243,0.005558,0.005390,0.004885,0.004548,0.004042,0.003706,0.003706,0.003200,0.003032,0.002358,0.002358,0.002190,0.002021,0.001853,0.001684,0.001516,0.001516,0.001516,0.001516,0.001516,0.001179,0.001179,0.001179,0.001179,0.001011,0.000842,0.000674,0.000674,0.000674,0.000674,0.000505,0.000505,0.000505,0.000505,0.000337,0.000337,0.000337,0.000337,0.000337,0.000337,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168
cumulative frequency,0.106114,0.210376,0.312110,0.406940,0.499916,0.581607,0.638201,0.684689,0.730504,0.765875,0.799057,0.824659,0.844703,0.862557,0.876874,0.886980,0.896918,0.906350,0.915277,0.922520,0.928078,0.933468,0.938353,0.942900,0.946943,0.950648,0.954354,0.957554,0.960586,0.962944,0.965302,0.967492,0.969513,0.971366,0.973050,0.974566,0.976082,0.977598,0.979114,0.980630,0.981809,0.982988,0.984167,0.985346,0.986357,0.987199,0.987873,0.988546,0.989220,0.989894,0.990399,0.990904,0.991410,0.991915,0.992252,0.992589,0.992926,0.993263,0.993599,0.993936,0.994105,0.994273,0.994442,0.994610,0.994779,0.994947,0.995115,0.995284,0.995452,0.995621,0.995789,0.995958,0.996126,0.996294,0.996463,0.996631,0.996800,0.996968,0.997137,0.997305,0.997473,0.997642,0.997810,0.997979,0.998147,0.998316,0.998484,0.998653,0.998821,0.998989,0.999158,0.999326,0.999495,0.999663,0.999832,1.000000


# We can see that by taking the first few brand origins up until Cyprus we already cover over 99% of the deals while not having to deal with multiple countries and small countries either, hence allowing a fast analysis of most orders and then leaving the corner cases for later

In [9]:
df.head(5).T

,0,1,2,3,4
hubspot_deal,46,5714,54,356,58
deal_value,1920,NaN,NaN,2400,NaN
brand_origin,portugal,sweden,portugal,singapore,spain
brand_tier,S,NaN,NaN,S,NaN
brand_type,Other,Adult,Other,Other,Men;Women
products_category,Accessories,Apparel,Accessories,Accessories,Footwear
production_origin,Portugal,"Portugal, Turkey",portugal,Portugal,spain
sales_channels,Ecommerce,Ecommerce;Wholesale;Marketplace,Ecommerce;Wholesale,Wholesale;Ecommerce,Ecommerce;Own Stores
instagram_followers,10455,NaN,5k - 20k,N.D.,> 100k
main_markets,Europe,Germany;Europe,NaN,Middle East,Europe


In [26]:
df

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''